# Imports

In [21]:
import nest_asyncio
import random

In [33]:
from pydantic_ai import Agent, RunContext, Tool

In [31]:
from pprint import pprint

# Set up asyncio


In [23]:
nest_asyncio.apply()


# Define the agent


In [24]:
agent = Agent(
    "openai:gpt-4o-mini",
    deps_type=str,  # This will be the name of the player
    system_prompt=(
        "You are a dice game, you should roll the die and see if the number "
        "you get back matches the user's guess. If so, tell them they're a winner. "
        "Tell them also the result of the roll. "
        "Use player's name in your response. To get player's name, use the tool `get_player_name`."
    ),
)

# Define the tools

## Registering tools via decorators

There are a number of ways to register tools with an agent:

- via the `@agent.tool` decorator — for tools that need access to the agent context
- via the `@agent.tool_plain` decorator — for tools that do not need access to the agent context
- via the `tools` keyword argument to Agent which can take either plain functions, or instances of Tool

`@agent.tool` is considered the default decorator since in the majority of cases tools will need access to the agent context.

### Tool plain (without context)

In [25]:
@agent.tool_plain
def roll_die() -> str:
    """Roll a six-sided die and return the result"""
    return str(random.randint(1, 6))


### Tool (with context)


In [26]:
@agent.tool
def get_player_name(ctx: RunContext[str]) -> str:
    """Get the player's name from the context"""
    return ctx.deps

### Run the agent


In [27]:
dice_result = agent.run_sync(
    user_prompt="My guess is 3",
    deps="John",
)

In [28]:
print(dice_result.data)

John, you guessed 3. Unfortunately, the roll was a 6, so you didn't win this time. Better luck next time!


### See what happened

In [32]:
pprint(dice_result.all_messages())

[ModelRequest(parts=[SystemPromptPart(content='You are a dice game, you should '
                                              'roll the die and see if the '
                                              'number you get back matches the '
                                              "user's guess. If so, tell them "
                                              "they're a winner. Tell them "
                                              'also the result of the roll. '
                                              "Use player's name in your "
                                              "response. To get player's name, "
                                              'use the tool `get_player_name`.',
                                      dynamic_ref=None,
                                      part_kind='system-prompt'),
                     UserPromptPart(content='My guess is 3',
                                    timestamp=datetime.datetime(2025, 3, 1, 8, 25, 40, 549849, tzinfo=date

## Registering tools via kwarg

### Simple way - list of functions

Just pass a list of functions to the `tools` kwarg.

In [35]:
agent_kwarged = Agent(
    "openai:gpt-4o-mini",
    tools=[roll_die, get_player_name],
)

In [38]:
result_kwarged = agent_kwarged.run_sync(
    user_prompt="My guess is 3",
    deps="John",
)



In [39]:
result_kwarged.data

'You guessed 3, but the die rolled a 5. Would you like to try again?'

### List of Tool objects


In [37]:
agent_kwarged_tools = Agent(
    "openai:gpt-4o-mini",
    tools=[
        Tool(roll_die, takes_ctx=False),  # Plain tool
        Tool(get_player_name, takes_ctx=True),  # Contextual tool
    ],
)

In [41]:
result_kwarged_tools = agent_kwarged_tools.run_sync(
    user_prompt="My guess is 3",
    deps="John",
)


In [42]:

result_kwarged_tools.data



'You guessed 3, but the die rolled a 6. Better luck next time! Would you like to play again or do something else?'